In [50]:
# Import necessary libraries
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [51]:
# Run only to generate random data (Ignore this cell for toy simulation)


dataset_length = 1000 #Length of the dataset (100 3D data points)

# x and y are the 2D dimensions of the equal dimension raster
x = 6
y = 6

#z is the time axis (input data point has 30 timestamps)
z = 30


data_points = []
for i in range(dataset_length):
    rasters = []
    for j in range(z):
        raster_value = np.random.randn()
        raster = [[raster_value]*y]*x #Each 2D raster has equal values
        rasters.append(raster)
    data_point = np.stack(tuple(rasters),axis = -1)
    data_points.append(data_point)
X = np.stack(tuple(data_points),axis = 0)


# Creating output rasters
z_out = 20
out_data_points = []
for i in range(dataset_length):
    out_rasters = []
    for j in range(z_out):
        out_raster_value = np.random.randn()
        out_raster = [[out_raster_value]*y]*x #Each 2D raster has equal values
        out_rasters.append(out_raster)
    out_data_point = np.stack(tuple(out_rasters),axis = -1)
    out_data_points.append(out_data_point)
X_out = np.stack(tuple(out_data_points),axis = 0)

print('Input size for dataset:', X.shape)
print('Output size for dataset:', X_out.shape)

# Split into train, validation, and test
X_train, X_test, y_train, y_test = train_test_split(X, X_out, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Z-score normalization
X_train = (X_train - X_train.mean())/X_train.std()
y_train = (y_train - y_train.mean())/y_train.std()
X_val = (X_val - X_train.mean())/X_train.std()
y_val = (y_val - y_train.mean())/y_train.std()
X_test = (X_test - X_train.mean())/X_train.std()
y_test = (y_test - y_train.mean())/y_train.std()

print('Training dataset input size:', X_train.shape)
print('Training dataset output size:', y_train.shape)
print('Validation dataset input size:', X_val.shape)
print('Validation dataset output size:', y_val.shape)
print('Test dataset input size:', X_test.shape)
print('Test dataset output size:', y_test.shape)

Input size for dataset: (1000, 6, 6, 30)
Output size for dataset: (1000, 6, 6, 20)
Training dataset input size: (600, 6, 6, 30)
Training dataset output size: (600, 6, 6, 20)
Validation dataset input size: (200, 6, 6, 30)
Validation dataset output size: (200, 6, 6, 20)
Test dataset input size: (200, 6, 6, 30)
Test dataset output size: (200, 6, 6, 20)


In [55]:
# Code to load and rescale Maththew's toy problem simulation data and create a train-validation-test split.

X = np.load(os.path.join('data', 'Xtest.npy'))
y = np.load(os.path.join('data', 'Ytest.npy'))

X = np.moveaxis(X,1,3)
y = np.moveaxis(y,1,3)

# Split into train, validation, and test
X_train, X_test, y_train, y_test = train_test_split(X, X_out, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Z-score normalization
X_train = (X_train - X_train.mean())/X_train.std()
y_train = (y_train - y_train.mean())/y_train.std()
X_val = (X_val - X_train.mean())/X_train.std()
y_val = (y_val - y_train.mean())/y_train.std()
X_test = (X_test - X_train.mean())/X_train.std()
y_test = (y_test - y_train.mean())/y_train.std()

print('Input size for dataset:', X.shape)
print('Output size for dataset:', y.shape)
print('Training dataset input size:', X_train.shape)
print('Training dataset output size:', y_train.shape)
print('Validation dataset input size:', X_val.shape)
print('Validation dataset output size:', y_val.shape)
print('Test dataset input size:', X_test.shape)
print('Test dataset output size:', y_test.shape)

Input size for dataset: (1000, 6, 6, 30)
Output size for dataset: (1000, 6, 6, 20)
Training dataset input size: (600, 6, 6, 30)
Training dataset output size: (600, 6, 6, 20)
Validation dataset input size: (200, 6, 6, 30)
Validation dataset output size: (200, 6, 6, 20)
Test dataset input size: (200, 6, 6, 30)
Test dataset output size: (200, 6, 6, 20)


In [61]:
def CNN2DModel(input_shape,
               output_shape,
               nFiltersList, 
               kernel_size=(2,2),
               opt=Adam(learning_rate=0.001), 
               maxpooling=False, # Not implemented yet 
               dropout=True, # Not implemented yet
               dropoutRate=0.5):
    '''
    CNN model using 2D convolutions
    
    nFiltersList is a list of # filters in each hidden layer.
    # layers is deduced from the length of nFiltersList
    '''
    
    inputs = Input(shape=input_shape)
    layer = inputs
    for i in range(len(nFiltersList)):
        layer = Conv2D(nFiltersList[i], kernel_size=(3,3), activation='relu', padding='same')(layer)
#         if maxpooling:
#             layer = MaxPooling2D((2, 2))(layer)
    
#     if dropout:
#         layer = Dropout(dropoutRate)(layer)

    outputs = Conv2D(output_shape[-1], kernel_size=kernel_size, activation='linear', padding='same')(layer)

    model = Model(inputs, outputs)
    model.compile(optimizer=opt, loss='mse', metrics=['mae'])

    return model

# Summary API
# plot diagnostic learning curves
def summarizeRegDiagnostics(history):
    fig, ax = plt.subplots(1,2, figsize=(20, 10))
    # plot loss
    ax[0].set_title('Loss Curves', fontsize=20)
    ax[0].plot(history.history['loss'], label='train')
    ax[0].plot(history.history['val_loss'], label='validation')
    ax[0].set_xlabel('Epochs', fontsize=15)
    ax[0].set_ylabel('Loss', fontsize=15)
    ax[0].legend(fontsize=15)
    # plot accuracy
    ax[1].set_title('MAE', fontsize=20)
    ax[1].plot(history.history['mae'], label='train')
    ax[1].plot(history.history['val_mae'], label='validation')
    ax[1].set_xlabel('Epochs', fontsize=15)
    ax[1].set_ylabel('MAE', fontsize=15)
    ax[1].legend(fontsize=15)

In [62]:
adam = Adam(learning_rate=0.01)
simple2DCNNModel = CNN2DModel(input_shape=X_train.shape[1:],
                              output_shape=y_train.shape[1:],
                              nFiltersList=[8, 16, 32, 64],
                              opt=adam)
simple2DCNNModel.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 6, 6, 30)]        0         
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 6, 6, 8)           2168      
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 6, 6, 16)          1168      
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 6, 6, 32)          4640      
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 6, 6, 64)          18496     
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 6, 6, 20)          5140      
Total params: 31,612
Trainable params: 31,612
Non-trainable params: 0
______________________________________________________

In [63]:
# Train
history = simple2DCNNModel.fit(X_train,y_train,
                               epochs = 500,
                               batch_size = 256,
                               validation_data=(X_val, y_val))
summarizeRegDiagnostics(history)

Train on 600 samples, validate on 200 samples
Epoch 1/500
600/600 [==============================] - 1s 2ms/sample - loss: 1.0200 - mae: 0.8062 - val_loss: 1.0302 - val_mae: 0.8124
Epoch 2/500
600/600 [==============================] - 0s 288us/sample - loss: 0.9990 - mae: 0.7986 - val_loss: 1.0305 - val_mae: 0.8124
Epoch 3/500
600/600 [==============================] - 0s 307us/sample - loss: 0.9985 - mae: 0.7985 - val_loss: 1.0308 - val_mae: 0.8124
Epoch 4/500
600/600 [==============================] - 0s 332us/sample - loss: 0.9987 - mae: 0.7987 - val_loss: 1.0309 - val_mae: 0.8123
Epoch 5/500
600/600 [==============================] - 0s 309us/sample - loss: 0.9980 - mae: 0.7986 - val_loss: 1.0309 - val_mae: 0.8121
Epoch 6/500
600/600 [==============================] - 0s 332us/sample - loss: 0.9980 - mae: 0.7986 - val_loss: 1.0307 - val_mae: 0.8120
Epoch 7/500
600/600 [==============================] - 0s 347us/sample - loss: 0.9979 - mae: 0.7985 - val_loss: 1.0307 - val_mae: 0.81

600/600 [==============================] - 0s 345us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0311 - val_mae: 0.8121
Epoch 61/500
600/600 [==============================] - 0s 320us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0310 - val_mae: 0.8122
Epoch 62/500
600/600 [==============================] - 0s 329us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0310 - val_mae: 0.8123
Epoch 63/500
600/600 [==============================] - 0s 319us/sample - loss: 0.9977 - mae: 0.7984 - val_loss: 1.0311 - val_mae: 0.8123
Epoch 64/500
600/600 [==============================] - 0s 328us/sample - loss: 0.9977 - mae: 0.7984 - val_loss: 1.0313 - val_mae: 0.8125
Epoch 65/500
600/600 [==============================] - 0s 330us/sample - loss: 0.9977 - mae: 0.7984 - val_loss: 1.0315 - val_mae: 0.8126
Epoch 66/500
600/600 [==============================] - 0s 286us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0317 - val_mae: 0.8127
Epoch 67/500
600/600 [=========================

600/600 [==============================] - 0s 327us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0309 - val_mae: 0.8121
Epoch 120/500
600/600 [==============================] - 0s 316us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0309 - val_mae: 0.8121
Epoch 121/500
600/600 [==============================] - 0s 332us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0310 - val_mae: 0.8121
Epoch 122/500
600/600 [==============================] - 0s 334us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0311 - val_mae: 0.8122
Epoch 123/500
600/600 [==============================] - 0s 332us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0312 - val_mae: 0.8122
Epoch 124/500
600/600 [==============================] - 0s 304us/sample - loss: 0.9977 - mae: 0.7984 - val_loss: 1.0313 - val_mae: 0.8122
Epoch 125/500
600/600 [==============================] - 0s 310us/sample - loss: 0.9977 - mae: 0.7984 - val_loss: 1.0313 - val_mae: 0.8123
Epoch 126/500
600/600 [==================

600/600 [==============================] - 0s 325us/sample - loss: 0.9978 - mae: 0.7983 - val_loss: 1.0315 - val_mae: 0.8123
Epoch 179/500
600/600 [==============================] - 0s 357us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0315 - val_mae: 0.8123
Epoch 180/500
600/600 [==============================] - 0s 335us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0314 - val_mae: 0.8123
Epoch 181/500
600/600 [==============================] - 0s 331us/sample - loss: 0.9977 - mae: 0.7984 - val_loss: 1.0313 - val_mae: 0.8124
Epoch 182/500
600/600 [==============================] - 0s 363us/sample - loss: 0.9978 - mae: 0.7985 - val_loss: 1.0313 - val_mae: 0.8124
Epoch 183/500
600/600 [==============================] - 0s 322us/sample - loss: 0.9977 - mae: 0.7985 - val_loss: 1.0314 - val_mae: 0.8125
Epoch 184/500
600/600 [==============================] - 0s 341us/sample - loss: 0.9978 - mae: 0.7985 - val_loss: 1.0314 - val_mae: 0.8125
Epoch 185/500
600/600 [==================

600/600 [==============================] - 0s 393us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0314 - val_mae: 0.8124
Epoch 238/500
600/600 [==============================] - 0s 384us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0314 - val_mae: 0.8124
Epoch 239/500
600/600 [==============================] - 0s 344us/sample - loss: 0.9978 - mae: 0.7985 - val_loss: 1.0312 - val_mae: 0.8123
Epoch 240/500
600/600 [==============================] - 0s 352us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0311 - val_mae: 0.8122
Epoch 241/500
600/600 [==============================] - 0s 363us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0311 - val_mae: 0.8122
Epoch 242/500
600/600 [==============================] - 0s 352us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0312 - val_mae: 0.8122
Epoch 243/500
600/600 [==============================] - 0s 358us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0313 - val_mae: 0.8123
Epoch 244/500
600/600 [==================

600/600 [==============================] - 0s 353us/sample - loss: 0.9977 - mae: 0.7984 - val_loss: 1.0318 - val_mae: 0.8125
Epoch 297/500
600/600 [==============================] - 0s 276us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0319 - val_mae: 0.8125
Epoch 298/500
600/600 [==============================] - 0s 321us/sample - loss: 0.9977 - mae: 0.7984 - val_loss: 1.0319 - val_mae: 0.8125
Epoch 299/500
600/600 [==============================] - 0s 341us/sample - loss: 0.9978 - mae: 0.7985 - val_loss: 1.0317 - val_mae: 0.8124
Epoch 300/500
600/600 [==============================] - 0s 354us/sample - loss: 0.9978 - mae: 0.7985 - val_loss: 1.0316 - val_mae: 0.8124
Epoch 301/500
600/600 [==============================] - 0s 321us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0313 - val_mae: 0.8123
Epoch 302/500
600/600 [==============================] - 0s 342us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0312 - val_mae: 0.8122
Epoch 303/500
600/600 [==================

600/600 [==============================] - 0s 350us/sample - loss: 0.9978 - mae: 0.7985 - val_loss: 1.0311 - val_mae: 0.8122
Epoch 356/500
600/600 [==============================] - 0s 342us/sample - loss: 0.9977 - mae: 0.7984 - val_loss: 1.0311 - val_mae: 0.8122
Epoch 357/500
600/600 [==============================] - 0s 355us/sample - loss: 0.9977 - mae: 0.7984 - val_loss: 1.0311 - val_mae: 0.8123
Epoch 358/500
600/600 [==============================] - 0s 343us/sample - loss: 0.9977 - mae: 0.7984 - val_loss: 1.0312 - val_mae: 0.8124
Epoch 359/500
600/600 [==============================] - 0s 361us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0315 - val_mae: 0.8125
Epoch 360/500
600/600 [==============================] - 0s 334us/sample - loss: 0.9977 - mae: 0.7984 - val_loss: 1.0316 - val_mae: 0.8125
Epoch 361/500
600/600 [==============================] - 0s 343us/sample - loss: 0.9978 - mae: 0.7985 - val_loss: 1.0317 - val_mae: 0.8125
Epoch 362/500
600/600 [==================

600/600 [==============================] - 0s 329us/sample - loss: 0.9978 - mae: 0.7985 - val_loss: 1.0318 - val_mae: 0.8124
Epoch 415/500
600/600 [==============================] - 0s 357us/sample - loss: 0.9977 - mae: 0.7985 - val_loss: 1.0318 - val_mae: 0.8124
Epoch 416/500
600/600 [==============================] - 0s 332us/sample - loss: 0.9978 - mae: 0.7985 - val_loss: 1.0317 - val_mae: 0.8123
Epoch 417/500
600/600 [==============================] - 0s 353us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0318 - val_mae: 0.8124
Epoch 418/500
600/600 [==============================] - 0s 353us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0318 - val_mae: 0.8123
Epoch 419/500
600/600 [==============================] - 0s 306us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0318 - val_mae: 0.8123
Epoch 420/500
600/600 [==============================] - 0s 350us/sample - loss: 0.9978 - mae: 0.7984 - val_loss: 1.0317 - val_mae: 0.8123
Epoch 421/500
600/600 [==================

KeyboardInterrupt: 